In [22]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [23]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Cambridge")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Cambridge', 'region': 'Cambridgeshire', 'country': 'United Kingdom', 'lat': 52.21, 'lon': 0.14, 'tz_id': 'Europe/London', 'localtime_epoch': 1718038869, 'localtime': '2024-06-10 18:01'}, 'current': {'last_updated_epoch': 1718038800, 'last_updated': '2024-06-10 18:00', 'temp_c': 10.0, 'temp_f': 50.0, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 5.6, 'wind_kph': 9.0, 'wind_degree': 340, 'wind_dir': 'NNW', 'pressure_mb': 1013.0, 'pressure_in': 29.91, 'precip_mm': 0.26, 'precip_in': 0.01, 'humidity': 87, 'cloud': 50, 'feelslike_c': 8.3, 'feelslike_f': 46.9, 'windchill_c': 10.4, 'windchill_f': 50.7, 'heatindex_c': 11.8, 'heatindex_f': 53.2, 'dewpoint_c': 8.9, 'dewpoint_f': 48.1, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 3.0, 'gust_mph': 10.8, 'gust_kph': 17.5}}"}, {'url': 'https://www.accuweather.com/en/gb/cambridge/cb1-1/june-weathe

In [24]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

In [25]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [26]:
model_with_tools = model.bind_tools(tools)

In [27]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [28]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_AqmxcacW0UW5EuBQRJ1NElI6'}]


In [29]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [30]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='c80f35dd-9449-463b-a1cd-63bfe258d0c1'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-befc0009-4c76-4ec5-b363-65e2c91c4e3a-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93})]

In [31]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the biggest news story in the uk and us currently?")]}
)
response["messages"]

[HumanMessage(content='whats the biggest news story in the uk and us currently?', id='a8a7ddaf-fc7e-4d7f-915f-fd13525ba342'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xooVJtwkczxYxQDxr0yXcxTd', 'function': {'arguments': '{\n  "query": "biggest news story in the UK"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 94, 'total_tokens': 119}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-fcb9593e-966d-4af3-88f9-0a31a749d834-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'biggest news story in the UK'}, 'id': 'call_xooVJtwkczxYxQDxr0yXcxTd'}], usage_metadata={'input_tokens': 94, 'output_tokens': 25, 'total_tokens': 119}),
 ToolMessage(content='[{"url": "https://www.bbc.co.uk/news/uk", "content": "Former Chancellor Alistair Darling dies aged 70\\nSacked TV boss slams chairman in gov

In [14]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zFwmQhKc57S6dAR3aRtkZE8E', 'function': {'arguments': '{\n  "query": "current weather in san francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 88, 'total_tokens': 112}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-430b453a-d457-4736-92e9-d2ee9f1c0d94-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in san francisco'}, 'id': 'call_zFwmQhKc57S6dAR3aRtkZE8E'}], usage_metadata={'input_tokens': 88, 'output_tokens': 24, 'total_tokens': 112})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'t

In [15]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

c:\Users\steph\OneDrive\Documents\UNIVERSITY\Personal_Projects\langchain_tutorials\.venv\lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


--
Starting tool: tavily_search_results_json with inputs: {'query': 'current weather in San Francisco'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1718029535, 'localtime': '2024-06-10 7:25'}, 'current': {'last_updated_epoch': 1718028900, 'last_updated': '2024-06-10 07:15', 'temp_c': 13.3, 'temp_f': 55.9, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 10, 'wind_dir': 'N', 'pressure_mb': 1015.0, 'pressure_in': 29.96, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 90, 'cloud': 75, 'feelslike_c': 12.7, 'feelslike_f': 54.9, 'windchill_c': 11.9, 'windchill_f': 53.5, 'heatindex_c': 12.6, 'heatindex_f': 54.6, 'dewpoint_c': 10

In [16]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [17]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [18]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 85, 'total_tokens': 96}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fe25b26b-b288-4dc4-834d-30b6cdd21d3b-0', usage_metadata={'input_tokens': 85, 'output_tokens': 11, 'total_tokens': 96})]}}
----


In [19]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob.', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 108, 'total_tokens': 114}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9e0d9dfd-21b8-4871-88aa-02f52b1dbb77-0', usage_metadata={'input_tokens': 108, 'output_tokens': 6, 'total_tokens': 114})]}}
----


In [20]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="As an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality.", response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 86, 'total_tokens': 125}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3235927e-e21b-4e70-affc-3068f469e398-0', usage_metadata={'input_tokens': 86, 'output_tokens': 39, 'total_tokens': 125})]}}
----
